# Make Diagonally Dominant Matrix

In [ ]:
import numpy as np


def is_diagonally_dominant(A):
    n = len(A)

    for i in range(n):
        # Check if the current row satisfies the diagonal dominance condition
        if abs(A[i, i]) < np.sum(np.abs(A[i, :])) - abs(A[i, i]):
            return False

    return True


def make_diagonally_dominant(A, b):
    n = len(A)

    # Reorder A and b to make A diagonally dominant
    for i in range(n):
        # Check if the current row satisfies the diagonal dominance condition
        if abs(A[i, i]) < np.sum(np.abs(A[i, :])) - abs(A[i, i]):
            # Find the row with the largest diagonal element in absolute value
            # Row with max diagonal element
            max_row = np.argmax(np.abs(A[i:, i])) + i

            # Swap the rows in A
            A[[i, max_row]] = A[[max_row, i]]

            # Swap the corresponding rows in b
            b[i], b[max_row] = b[max_row], b[i]

    return A, b


# Example usage:
A = np.array([[1, -2, 3],
              [2,  1, -1],
              [3,  2, 1]], dtype=float)

b = np.array([3, -1, 2], dtype=float)

A_dominant, b_dominant = make_diagonally_dominant(A, b)

print("Reordered matrix A:\n", A_dominant)
print("Reordered vector b:\n", b_dominant)

print("Is A diagonally dominant now?", is_diagonally_dominant(A_dominant))

Reordered matrix A:
 [[ 3.  2.  1.]
 [ 1. -2.  3.]
 [ 2.  1. -1.]]
Reordered vector b:
 [ 2.  3. -1.]
Is A diagonally dominant now? False


# Jacobi Iterative Method


In [ ]:
import numpy as np


def jacobi(A, b, tol=1e-10, max_iter=100):
    """
    Solves Ax = b using the Jacobi Iterative Method.

    Parameters:
    - A: Coefficient matrix (should be square and diagonally dominant).
    - b: Right-hand side vector.
    - tol: Tolerance for stopping criterion.
    - max_iter: Maximum number of iterations.

    Returns:
    - x: Solution vector.
    """
    # Ensure A is square
    n, m = A.shape
    if n != m:
        raise ValueError("Matrix A must be square.")

    # Ensure diagonal dominance (or warn the user)
    for i in range(n):
        if abs(A[i, i]) < sum(abs(A[i, j]) for j in range(n) if j != i):
            raise ValueError(
                "Matrix A must be diagonally dominant for guaranteed convergence.")

    # Initialize solution vector (start with zeros)
    x = np.zeros_like(b, dtype=float)
    x_new = np.zeros_like(b, dtype=float)

    # Iterative process
    for iteration in range(max_iter):
        for i in range(n):
            # Update x[i] using the Jacobi formula
            s = np.sum([A[i, j] * x[j] for j in range(n) if j != i])
            x_new[i] = (b[i] - s) / A[i, i]

        # Check for convergence
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            print(f"Converged in {iteration + 1} iterations.")
            return x_new

        # Update x for the next iteration
        x = x_new.copy()

    # If no convergence after max_iter
    raise ValueError("Jacobi method did not converge within the maximum number of iterations.")


# Example usage
A = np.array([[10, -1, 2],
              [-1, 11, -1],
              [2, -1, 10]], dtype=float)
b = np.array([6, 25, -11], dtype=float)

try:
    solution = jacobi(A, b)
    print("Solution:", solution)
except ValueError as e:
    print(e)

Converged in 19 iterations.
Solution: [ 1.04326923  2.26923077 -1.08173077]


# Gauss-Seidel Iterative Method


In [ ]:
import numpy as np


def gauss_seidel(A, b, tol=1e-10, max_iter=100):
    """
    Solves Ax = b using the Gauss-Seidel Iterative Method.

    Parameters:
    - A: Coefficient matrix (should be square and preferably diagonally dominant).
    - b: Right-hand side vector.
    - tol: Tolerance for stopping criterion.
    - max_iter: Maximum number of iterations.

    Returns:
    - x: Solution vector.
    """
    # Ensure A is square
    n, m = A.shape
    if n != m:
        raise ValueError("Matrix A must be square.")

    # Ensure diagonal dominance (or warn the user)
    for i in range(n):
        if abs(A[i, i]) < sum(abs(A[i, j]) for j in range(n) if j != i):
            raise ValueError(
                "Matrix A must be diagonally dominant for guaranteed convergence.")

    # Initialize solution vector (start with zeros)
    x = np.zeros_like(b, dtype=float)

    # Iterative process
    for iteration in range(max_iter):
        x_new = np.copy(x)  # Make a copy to store new values
        for i in range(n):
            # Update x[i] using the Gauss-Seidel formula
            s1 = sum(A[i, j] * x_new[j] for j in range(i))     # Use updated values
            s2 = sum(A[i, j] * x[j] for j in range(i + 1, n))  # Use old values
            x_new[i] = (b[i] - s1 - s2) / A[i, i]

        # Check for convergence
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            print(f"Converged in {iteration + 1} iterations.")
            return x_new

        # Update x for the next iteration
        x = x_new.copy()

    # If no convergence after max_iter
    raise ValueError("Gauss-Seidel method did not converge within the maximum number of iterations.")


# Example usage
A = np.array([[10, -1, 2],
              [-1, 11, -1],
              [2, -1, 10]], dtype=float)
b = np.array([6, 25, -11], dtype=float)

try:
    solution = gauss_seidel(A, b)
    print("Solution:", solution)
except ValueError as e:
    print(e)

Converged in 10 iterations.
Solution: [ 1.04326923  2.26923077 -1.08173077]


# Gauss Elimination


In [25]:
import numpy as np


def gauss_elimination(A, b):
    """
    Solves Ax = b using the Gauss Elimination Method.

    Parameters:
    - A: Coefficient matrix (should be square and non-singular).
    - b: Right-hand side vector.

    Returns:
    - x: Solution vector.
    """
    # Ensure A is square
    n, m = A.shape
    if n != m:
        raise ValueError("Matrix A must be square.")

    # Augment matrix A with vector b
    Ab = np.hstack((A, b.reshape(-1, 1)))

    # Forward Elimination
    for i in range(n):
        # Check for zero pivot (swap rows if needed)
        if Ab[i, i] == 0:
            for j in range(i + 1, n):
                if Ab[j, i] != 0:
                    Ab[[i, j]] = Ab[[j, i]]  # Swap rows
                    break
            else:
                raise ValueError("Matrix is singular or nearly singular.")

        # Normalize the pivot row
        Ab[i] = Ab[i] / Ab[i, i]

        # Eliminate entries below the pivot
        for j in range(i + 1, n):
            Ab[j] -= Ab[j, i] * Ab[i]

    # Back Substitution
    x = np.zeros(n, dtype=float)
    for i in range(n - 1, -1, -1):
        x[i] = Ab[i, -1] - np.sum(Ab[i, i + 1:n] * x[i + 1:n])

    return x


# Example usage
A = np.array([[2, -1, 1],
              [3, 3, 9],
              [3, 3, 5]], dtype=float)
b = np.array([2, -1, 1], dtype=float)

try:
    solution = gauss_elimination(A, b)
    print("Solution:", solution)
except ValueError as e:
    print(e)

Solution: [ 1.22222222 -0.05555556 -0.5       ]


# Gauss-Jordan Elimination


In [4]:
import numpy as np


def gauss_jordan_elimination(A, b):
    """
    Solves Ax = b using the Gauss-Jordan Elimination Method.

    Parameters:
    - A: Coefficient matrix (should be square and non-singular).
    - b: Right-hand side vector.

    Returns:
    - x: Solution vector.
    """
    # Ensure A is square
    n, m = A.shape
    if n != m:
        raise ValueError("Matrix A must be square.")

    # Augment matrix A with vector b
    Ab = np.hstack((A, b.reshape(-1, 1)))

    # Forward and Backward Elimination
    for i in range(n):
        # Check for zero pivot (swap rows if needed)
        if Ab[i, i] == 0:
            for j in range(i + 1, n):
                if Ab[j, i] != 0:
                    Ab[[i, j]] = Ab[[j, i]]  # Swap rows
                    break
            else:
                raise ValueError("Matrix is singular or nearly singular.")

        # Normalize the pivot row
        Ab[i] = Ab[i] / Ab[i, i]

        # Eliminate all entries in the current column except the pivot
        for j in range(n):
            if j != i:
                Ab[j] -= Ab[j, i] * Ab[i]

    # Extract solution from the last column
    x = Ab[:, -1]
    return x


# Example usage
A = np.array([[2, -1, 1],
              [3, 3, 9],
              [3, 3, 5]], dtype=float)
b = np.array([2, -1, 1], dtype=float)

try:
    solution = gauss_jordan_elimination(A, b)
    print("Solution:", solution)
except ValueError as e:
    print(e)

Solution: [ 1.22222222 -0.05555556 -0.5       ]


# LU Factorization


In [5]:
import numpy as np


def lu_factorization(A):
    """
    Performs LU factorization using Doolittle's method.

    Parameters:
    - A: Coefficient matrix (must be square and non-singular).

    Returns:
    - L: Lower triangular matrix.
    - U: Upper triangular matrix.
    """
    n, m = A.shape
    if n != m:
        raise ValueError("Matrix A must be square.")

    # Initialize L and U
    L = np.eye(n)  # Lower triangular matrix (identity matrix initially)
    U = np.zeros_like(A)  # Upper triangular matrix

    for i in range(n):
        # Fill U's row i
        for j in range(i, n):
            U[i, j] = A[i, j] - sum(L[i, k] * U[k, j] for k in range(i))

        # Fill L's column i
        for j in range(i + 1, n):
            if U[i, i] == 0:
                raise ValueError("Matrix is singular or nearly singular.")
            L[j, i] = (A[j, i] - sum(L[j, k] * U[k, i]
                       for k in range(i))) / U[i, i]

    return L, U


def solve_lu(L, U, b):
    """
    Solves Ax = b using forward and backward substitution with L and U matrices.

    Parameters:
    - L: Lower triangular matrix.
    - U: Upper triangular matrix.
    - b: Right-hand side vector.

    Returns:
    - x: Solution vector.
    """
    n = len(b)

    # Forward substitution to solve Lc = b
    c = np.zeros_like(b, dtype=float)
    for i in range(n):
        c[i] = b[i] - sum(L[i, j] * c[j] for j in range(i))

    # Backward substitution to solve Ux = c
    x = np.zeros_like(b, dtype=float)
    for i in range(n - 1, -1, -1):
        x[i] = (c[i] - sum(U[i, j] * x[j] for j in range(i + 1, n))) / U[i, i]

    return x


# Example usage
A = np.array([[2, -1, 1],
              [3, 3, 9],
              [3, 3, 5]], dtype=float)
b = np.array([2, -1, 1], dtype=float)

try:
    L, U = lu_factorization(A)
    print("L:\n", L)
    print("U:\n", U)
    solution = solve_lu(L, U, b)
    print("Solution:", solution)
except ValueError as e:
    print(e)

L:
 [[1.  0.  0. ]
 [1.5 1.  0. ]
 [1.5 1.  1. ]]
U:
 [[ 2.  -1.   1. ]
 [ 0.   4.5  7.5]
 [ 0.   0.  -4. ]]
Solution: [ 1.22222222 -0.05555556 -0.5       ]


# Matrix Inversion using Gauss-Jordan Elimination


In [7]:
import numpy as np


def matrix_inverse(A):
    """
    Computes the inverse of a square matrix A using Gauss-Jordan elimination.

    Parameters:
    - A: Coefficient matrix (must be square and non-singular).

    Returns:
    - A_inv: Inverse of matrix A.
    """
    n, m = A.shape
    if n != m:
        raise ValueError("Matrix A must be square.")

    # Augment A with the identity matrix
    I = np.eye(n)
    Ab = np.hstack((A, I))

    # Apply Gauss-Jordan elimination
    for i in range(n):
        # Check for zero pivot and swap rows if necessary
        if Ab[i, i] == 0:
            for j in range(i + 1, n):
                if Ab[j, i] != 0:
                    Ab[[i, j]] = Ab[[j, i]]  # Swap rows
                    break
            else:
                raise ValueError("Matrix is singular or nearly singular.")

        # Normalize the pivot row
        Ab[i] = Ab[i] / Ab[i, i]

        # Eliminate all entries in the current column except the pivot
        for j in range(n):
            if j != i:
                Ab[j] -= Ab[j, i] * Ab[i]

    # Extract the inverse matrix from the augmented matrix
    A_inv = Ab[:, n:]
    return A_inv


# Example usage
A = np.array([[4, 7],
              [2, 6]], dtype=float)

try:
    A_inv = matrix_inverse(A)
    print("Inverse of A:\n", A_inv)
    print("Verification (A * A_inv):\n", np.round(np.dot(A, A_inv), 6))
except ValueError as e:
    print(e)

Inverse of A:
 [[ 0.6 -0.7]
 [-0.2  0.4]]
Verification (A * A_inv):
 [[ 1. -0.]
 [ 0.  1.]]
